In [9]:
#importing the libraries and the dataframe
import pandas as pd
import numpy as np
import json


filepath = 'source_file_2.json'
priorities = pd.read_json(filepath)
priorities.head()

,name,managers,watchers,priority
0,[CV] [Qt] OpenCV GUI,"[csaftoiu, merlin]","[merlin, morris]",850
1,[Web] pm-site,[merlin],"[csaftoiu, jpfarias, morris]",260
2,Annotations,[luanda],"[csaftoiu, merlin, victor]",10
3,[OCR] tesseract MRZ reading,[csaftoiu],"[arturc, merlin]",560
4,Deepgammon,[morris],"[merlin, victor]",150


In [10]:
#Cleaning the text in the manager and watchers columns
col_list = ['managers','watchers']

for col in col_list:
    priorities.loc[:,col] = priorities.loc[:,col].astype(str).str.strip('[]')
    priorities.loc[:,col] = priorities.loc[:,col].str.replace("'",'') 
    priorities.loc[:,col] = priorities.loc[:,col].str.replace(" ",'') 

priorities

,name,managers,watchers,priority
0,[CV] [Qt] OpenCV GUI,"csaftoiu,merlin","merlin,morris",850
1,[Web] pm-site,merlin,"csaftoiu,jpfarias,morris",260
2,Annotations,luanda,"csaftoiu,merlin,victor",10
3,[OCR] tesseract MRZ reading,csaftoiu,"arturc,merlin",560
4,Deepgammon,morris,"merlin,victor",150
5,python training lucas,merlin,"csaftoiu,morris",2160
6,python training alysson,merlin,"csaftoiu,morris",2180
7,unity render different passport text,csaftoiu,"merlin,morris",2230
8,[template] unity render passport text,csaftoiu,"merlin,morris",2235
9,unity render diff passport text artur,"arturc,csaftoiu","merlin,morris",815


In [11]:
#creating new dataframes one for the managers and the other for the watchers
col_watchers = 'watchers'
col_managers = 'managers'
managers = priorities.loc[:,priorities.columns != col_watchers]
watchers = priorities.loc[:,priorities.columns != col_managers]

In [12]:
#credits to https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

def explode(df, lst_cols, fill_value='', preserve_index=False):
    '''
    Takes in a dataframe and column list and separates the string in each column 
    into new rows with the same values per each string found (separator needs to be passed) value found.
    '''
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

#separarting the managers into new rows and ordering by priority
managers_exploded = explode(managers.assign(managers=managers.managers.str.split(',')), 'managers').sort_values(by='priority')

managers_exploded

,name,priority,managers
3,Annotations,10,luanda
23,[EXECUTIVE] Merlin,20,merlin
49,sportsparsers - 22bet,45,morris
48,sportsparsers - esportscalendar,55,morris
36,morris bg data gathering,130,victor
46,sportsparsers - unikrn,145,morris
5,Deepgammon,150,morris
45,Runningball Parser,155,morris
47,sportsparsers - unibet,250,morris
2,[Web] pm-site,260,merlin


In [13]:
#same but for watchers
watchers_exploded = explode(watchers.assign(watchers=watchers.watchers.str.split(',')), 'watchers').sort_values(by='priority')

watchers_exploded

,name,priority,watchers
5,Annotations,10,csaftoiu
6,Annotations,10,merlin
7,Annotations,10,victor
47,[EXECUTIVE] Merlin,20,jpfarias
48,[EXECUTIVE] Merlin,20,merlin
...,...,...,...
91,[INHOUSE-TOOLS] NN-Webapp 2,2520,merlin
90,[INHOUSE-TOOLS] NN-Webapp 2,2520,jpfarias
89,[INHOUSE-TOOLS] NN-Webapp 2,2520,csaftoiu
98,matchbook parser - mfilho,2575,merlin


In [14]:
#grouping by projects by manager, ordering by priority and transforming into dictionary
dict_managers = managers_exploded.groupby(['managers'])['name'].apply(lambda grp: list(grp.value_counts().index)).to_dict()
dict_managers

{'ariel': ['[BG-CLUB-SITE] frontend'],
 'arturc': ['unity render diff passport text artur'],
 'csaftoiu': ['[template] unity render passport text',
  '[OCR] tesseract MRZ reading ',
  'unity render different passport text',
  '[CV] [Qt] OpenCV GUI',
  'unity render diff passport text artur',
  'sportsparsers - pinnacle API',
  'matchbook parser - mfilho'],
 'luanda': ['Annotations'],
 'merlin': ['python training alysson',
  'python training lucas',
  '[CHATBOT] rodrigo',
  '[INHOUSE-TOOLS] Annotators app',
  '[ID-VERIFY-PIPELINE] Passport finder',
  '[HR] Candidate tests',
  '[IOS-APPS] IOS PI app',
  'Unity Render Passport Text [Template]',
  'chess board finder eliaquim',
  '[EXECUTIVE] Merlin',
  '[CV] [Qt] OpenCV GUI',
  'unity render diff passport text - kennedy',
  '[CHESS] Live Demo Executive',
  '[INHOUSE-TOOLS] NN-Webapp 2',
  '[CHATBOT] merlin',
  '[Web] pm-site',
  '[ID-VERIFY-PIPELINE] Background evaluator',
  '[INHOUSE-TOOLS] NN-Webapp',
  '[IOS-APPS] IOS streaming app',
 

In [15]:
#same for watchers
dict_watchers = watchers_exploded.groupby(['watchers'])['name'].apply(lambda grp: list(grp.value_counts().index)).to_dict()
dict_watchers

{'Alex': ['sportsparsers - pinnacle API'],
 'arturc': ['[OCR] tesseract MRZ reading '],
 'csaftoiu': ['morris bg data gathering',
  '[ID-VERIFY-PIPELINE] finger detector',
  'sportsparsers - betway',
  'sportsparsers - gg.bet',
  'Annotations',
  'python training lucas',
  'sportsparsers - 22bet',
  '[EXECUTIVE] Merlin',
  'sportsparsers - tipster.de',
  'python training alysson',
  'tourney site famador',
  '[HR] Candidate tests',
  '[ID-VERIFY-PIPELINE] Passport finder',
  'sportsparsers - unibet',
  '[INHOUSE-TOOLS] Annotators app',
  '[CHATBOT] rodrigo',
  '[CHATBOT] merlin',
  'unity render diff passport text - kennedy',
  '[ID-VERIFY-PIPELINE] Background evaluator',
  'bg deep checkers rvalentim',
  'bg deep learning aporto',
  '[CHESS] Live Demo Executive',
  'mateus silva bg deep checkers',
  '[BG-CLUB-SITE] frontend',
  'sportsparsers - esportscalendar',
  '[INHOUSE-TOOLS] NN-Webapp',
  'tourney site tbarbalho',
  'Android Camera App rrocha',
  '[Web] pm-site',
  'chess board 

In [16]:
#exporting the newly created dictionaries into json files
with open('managers.json', 'w') as json_file:
    json.dump(dict_managers, json_file)
    
with open('watchers.json', 'w') as json_file:
    json.dump(dict_watchers, json_file)    